In [1]:
# ПОИСК НОВОСТЕЙ О РЕЗУЛЬТАТАХ ИСПОЛНЕНИЯ БЮДЖЕТА КНР


import requests
import os
import re
import pandas as pd

import openpyxl

from pandas import ExcelWriter
from bs4 import BeautifulSoup

In [11]:
#function to find positions in a string
def get_positions(text,s_string):
    
    str_pos=text.find(s_string)
        
    for i in range(10):
        if not text[str_pos-i-1].isdigit():
            num_pos1=str_pos-i
            break
        
        
    dgt_flag=True
        
    if '下降' in text[str_pos:]:
        dgt_sign='-'
    else:
        dgt_sign=''
                
        
    prct_scale=True
    
    
    if '持平' not in text[str_pos:]:

        for j in range(10):
            if dgt_flag and text[str_pos+len(s_string)+j+1].isdigit():
                dgt_flag=False
                num_pos2=str_pos+len(s_string)+j+1

            if text[str_pos+len(s_string)+j+1] == '%' or text[str_pos+len(s_string)+j+1] == '倍':
                num_pos3=str_pos+len(s_string)+j+1
                if s_item[str_pos+len(s_string)+j+1] == '倍':
                    prct_scale=False
                break 
    
    stop_list=['；','.','；'] 
    stop_pos=0
    for stop_item in stop_list:
        if stop_item in text[:num_pos1]:
            stop_pos=text[:num_pos1].find(stop_item)+1
            break
    
    bgt_item=text[stop_pos:num_pos1]
        
    if bgt_item.find('，')>=0:
        bgt_item=bgt_item.split('，')[1]
        
    bgt_value=int(text[num_pos1:str_pos])
    
    if '持平' in text[str_pos:]:
        bgt_grt = 0
    else:
        bgt_grt=dgt_sign+text[num_pos2:num_pos3]
    
    
    if prct_scale:
        bgt_grt_num=float(bgt_grt)
    else:
        bgt_grt_num=round(float(bgt_grt)*100,1)
        
        
    # returns data about revenues and expenditures 
        
    return bgt_item,bgt_value,bgt_grt_num

In [3]:
# path at workplace
my_path='C:\\Users\\plehanov\\Documents\\3_TELEGRAM\\China\\CHN_Pyton\\MOF_DATA'

# path at home
# my_path='C:\\Users\\dima\\Documents\\8_Data_Analysis\\MOF_DATA'

my_filename='MOF_data.xlsx'
    

    

In [4]:
def save_data_xls(data_list,data_period,path,filename):

    labels=['period','item','value','g_rate']
    df = pd.DataFrame.from_records(data_list, columns=labels)
    
    sheet_name=data_period
    
    file=os.path.join(path,filename)
    
    writer = ExcelWriter(file, engine='openpyxl')

    if os.path.exists(file):
        book = openpyxl.load_workbook(file)
        writer.book = book
        
        sheets=book.sheetnames
        #print(sheets[5])
        if sheet_name in sheets:
            book.remove(book[sheet_name])
            
    df.to_excel(writer, sheet_name,index=False)
    writer.save()
    writer.close()
    
    
    

In [5]:

# лист для сохранения ссылок на новости
link_list=list()

for step in range(1):
    
    if step ==0:
        page_link='http://www.mof.gov.cn/zhengwuxinxi/caizhengshuju/'
        # остальные страницы с новостями получаются добавлением адреса index_1.htm, index_2.htm и т.д.
    else:
        page_link='http://www.mof.gov.cn/zhengwuxinxi/caizhengshuju/index_'+str(step)+'.htm'
        
    response=requests.get(page_link)
    result_page=BeautifulSoup(response.content,'lxml')

    #распечатать результаты запроса
    print(result_page.prettify())
    
    # ПОИСК ВСЕХ НОВОСТЕЙ ПО БЮДЖЕТУ НА СТРАНИЦЕ
    #   типичное название новости
    #   2019年1-2月财政收支情况 
    # строка для поиска
    search_string='财政收支情况'
    # лист для сохранения ссылок

    table_tag=result_page.find('table', class_="ZIT")
    #print(table_tags)

    td_tags = table_tag.find_all('td')

    for td_item in td_tags:
        if search_string in td_item.get('title'):
            news_head=td_item.get('title')
            news_link=td_item.find('a').get('href')
            
            
            # в некоторых случаях неполный адрес указывается
            # приходится добавлять в начало часть адреса
            if news_link.startswith('..'):
                news_link='http://gks.mof.gov.cn/zhengfuxinxi' + news_link[2:]
            
            link_list.append([news_head,news_link])
        

print(link_list)    

    

<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">
<html xmlns="http://www.w3.org/1999/xhtml">
 <head>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <title>
   财政数据
  </title>
  <link href="../../images/ben-css_tylb.css" rel="stylesheet" type="text/css"/>
  <link href="../../images/czb_index_20170824.css" rel="stylesheet" type="text/css"/>
  <script src="../../images/bjdflash.js" type="text/javascript">
  </script>
  <style>
   html,body{
         overflow:hidden;
         margin:0px;
         width:100%;
         height:100%;
}

#new_body{
         overflow-y:scroll; 
         overflow-x:auto;        
         width:100%;
         height:100%;
}

#guding{
        position:absolute;
        z-index:2008;
        top:270px;
        width:120px;
        height:140px; 
}
  </style>
 </head>
 <body>
  <div id="new_body">
   <!--tou-->
   <script language="JavaScript">
    var i=0
   </script>


In [12]:
# по каждому новостному сообщению осуществляется переход на страницу, сбор данных о статьях и величине расходов и доходов,
# а также темпах роста показателей

# затем эти данные сохраняются на листе в файле Excel
# список для сохранения данных о бюджете
#save_list=list()

for link_item in link_list:

    
    page_link=link_item[1]
    data_str=link_item[0]
    print(data_str)
    period=data_str[:data_str.find('财政')]


    response=requests.get(page_link)
    result_page=BeautifulSoup(response.content,'lxml')
    
    div_tags=result_page.find_all('div')
    news_text=div_tags[1].get_text()


    # multiple split
    # break test into list by delimiters '。' OR '；'

    #s_text=news_text.split('。')

    s_text=re.split('。|；|其中',news_text)

    #print(s_text)



    sstring='亿元，同比'
    stop_list=['；','.','；']
    save_list=list()
    for s_item in s_text:
        if sstring in s_item:
            print(s_item)
            text,value,g_rate = get_positions(s_item,sstring)
            
            save_list.append([period,text,value,g_rate])
    
    save_data_xls(save_list,period,my_path,my_filename)

        
        
print(save_list)

        

        
        
        
            
        
            
    

2019年前三季度财政收支情况  （2019-10-17）

　　1-9月累计，全国一般公共预算收入150678亿元，同比增长3.3%
，中央一般公共预算收入72045亿元，同比增长3.5%
地方一般公共预算本级收入78633亿元，同比增长3.1%
全国一般公共预算收入中的税收收入126970亿元，同比下降0.4%
非税收入23708亿元，同比增长29.2%

　　1-9月累计，主要税收收入项目情况如下：
　　1.国内增值税49336亿元，同比增长4.2%

　　2.国内消费税11448亿元，同比增长15.8%

　　3.企业所得税31572亿元，同比增长2.7%

　　4.个人所得税7981亿元，同比下降29.7%

　　5.进口货物增值税、消费税12148亿元，同比下降8.1%
关税2149亿元，同比下降3%

　　6.出口退税12860亿元，同比增长12.8%

　　7.城市维护建设税3696亿元，同比持平

　　8.车辆购置税2674亿元，同比下降0.4%

　　9.印花税1989亿元，同比增长10%
，证券交易印花税1072亿元，同比增长21.7%

　　10.资源税1407亿元，同比增长11.9%

　　11.土地和房地产相关税收中，契税4800亿元，同比增长6.3%
土地增值税5077亿元，同比增长10.8%
房产税1970亿元，同比增长0.2%
耕地占用税1098亿元，同比增长7%
城镇土地使用税1528亿元，同比下降13.2%

　　12.环境保护税168亿元，同比增长69.2%

　　13.车船税、船舶吨税、烟叶税等其他各项税收收入合计790亿元，同比增长11.1%

　　1-9月累计，全国一般公共预算支出178612亿元，同比增长9.4%
，中央一般公共预算支出25079亿元，同比增长9.2%
地方一般公共预算本级支出153533亿元，同比增长9.4%

　　1-9月累计，主要支出科目情况如下：
　　1.教育支出26054亿元，同比增长9.7%

　　2.科学技术支出6013亿元，同比增长10.6%

　　3.文化旅游体育与传媒支出2572亿元，同比增长4.8%

　　4.社会保障和就业支出23897亿元，同比增长9.2%

　　5.卫生健康支出13788亿元，同比增长10%

　　6.节能环保支出4750亿元，同比增长14.5%

　　7.城